<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/MCDWT-experimentos/blob/master/src/quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comprimiendo subbandas

Os digo lo que creo que podríamos hacer, dependiendo de la escalabilidad espacio/temporal requerida. Más adelante discutimos la escalabilidad por calidad.

### Comprimiendo directamente $L^0_0$ (vídeo original)
Si estamos comprimiendo $L^0_0$ es que no hemos aplicado PRMC. Podemos usar cualquier compresor de imágenes/vídeo, escalable o no, dependiendo de lo que necesitemos (aunque seguramente en modo no intra). Je, aquí me he reventado la cabeza pensando ...

### Comprimiendo $L^1_0$ y $H^1_{-1}$
Conseguir esta forma de escalabiliad únicamente temporal es un poco lioso con el código que hay desarrollado ahora mismo, pero posible si previamente a usar PRMC(1) interpolamos el vídeo con la DWT (usada para conseguir la OLP) para conseguir $L^0_{-1}$. De esta manera vamos a obtener las secuencias $L^1_0$ y $H^1_{-1}$ en las que las subbandas espaciales H (de ambas secuencias) son siempre 0, y por tanto se pueden ignorar. Cada frame (LL) de $H^1_{-1}$ debería comprimirse en modo intra y $L^1_0$ en modo no intra (dentro de cada GOP). El bit-allocation dentro de $L^1_0$ estaría controlado por el compresor de vídeo usado. Dentro del mismo GOP, el bit-allocation para las subbandas LL de $H^1_{-1}$ debería generar el mismo slope R/D que el que se ha conseguido para las subbandas LL de $L^1_0$.

### Comprimiendo $L^1_1$ y $H^1_0$
Este es el resultado de aplicar PRMC(1) a $L^0_0$, generándose 2 subbandas temporales: $L^1_1$ y $H^1_0$, la primera de resolución $Y/2\times X/2$ y la segunda proporcionando la resolución $Y\times X$. La subbanda temporal $L^1_1$ se debería comprimir en modo no intra, siendo $\lambda$ el R/D slope alcanzado, que si no me equivoco se podría calcular como (usando el MSE como medida de la distorsión) 
$$
    \lambda=\frac{\text{MSE}_\text{gray} - \text{MSE}}{\text{bit-rate}},
$$
donde $\text{MSE}_\text{gray}$ sería el resultado de comparar la secuencia "gris" (con todas sus componentes iguales a 128) con $L^1_1$, y $\text{MSE}$ resultaría de comparar $\hat{L}^1_1$ con $L^1_1$. $\text{bit-rate}$ es el bit-rate de la secuencia $\hat{L}^1_1$ comprimida.

Usando dicho $\lambda$, las subbandas espaciales de alta frecuencia de $H^1_0$ se deberían comprimir en 3 secuencias de vídeo independientes (una con las subbandas $LH$, otra con las subbandas $HL$ y otra con las subbandas $HH$) de manera que el slope total proporcionado fuera
$$
\lambda = \alpha\lambda'
$$
donde $\lambda'$ es el slope "local" generado, calculado como
$$
    \lambda'=\frac{\text{MSE}_\text{zero} - \text{MSE}}{\text{bit-rate}},
$$
donde $\text{MSE}_\text{zero}$ sería el resultado de comparar $\hat{H}^1_0$ con la secuencia "zero" (con todas sus componentes iguales a 0) y $\text{MSE}$ resultaría de comparar $\hat{H}^1_0$ con $H^1_0$. $\text{bit-rate}$ es el bit-rate total para las 3 secuencias comprimidas de $\hat{H}^1_0$.

## Cuantificando ... o comprimiendo con pérdida

La DWT, al usar "orthogonal basis functions" produce subbandas independientes, lo que significa que la contribución de una subbanda a la calidad de la reconstrucción no depende del contenido del resto de subbandas. Cuando aplicamos MCOLP la cosa cambia, porque en este caso existe una dependencia entre subbandas temporales. Analicemos algunos casos:

### Si no usamos MCOLP ...

Este caso ya está resuelto.

Las subbandas hay que 
... o comprimiendo, pero controlando la contribución de cada subbanda.

Dentro de un frame transformado, las subbandas no contribuyen en la misma proporción cuando se trata de reconstruir el frame (casi ninguna DWT es orthonormal ... lo escribo en inglés, porque en español me suena raro). Y dentro de una secuencia transformada, las subbandas temporales tampoco son igualmente importantes de cara a disminuir nuestro amado/odiado R/D tradeoff (las subbandas de menor frecuencia normalmente contribuyen más). De hecho en MCOLP, la transformada no es ni biorthogonal, lo que significa que lo que contribuye una subbanda temporal $H^x$ depende de lo que contribuyen las subbandas $H^{x+1}$, $H^{x+2}$ y por supuesto, la subbanda $L^y$ (siendo $y\geq x+2$).

Supongamos que aplicamos una MCOLP(3) a un vídeo, lo que significa que generamos 2 niveles de resolución espacial y 4 niveles de resolución temporal, generándose la subbandas $L^3_1, H^3_1, H^2_1, H^1_1, L^3_0, H^3_0, H^2_0, H^1_0$.

Aquí es donde está la madre del cordero (en controlar cuánto se cuantifican las diferentes subbandas) y donde veo yo el trabajo más duro. La solución más simple, y que en muchos casos funcionará más o menos bien, es suponer que MCOLP es al menos biorthogonal y por lo tanto, simplemente asignándole un peso adecuado a cada subbanda podemos cuantificar, y de ahí, pasar a comprimir.

Sin embargo, aunque el problema del rate-allocation parece complejo, podemos (creo yo) atacarlo suponiendo como en todo este proceso, que la reconstrucción del vídeo se va a producir por niveles de resolución (espaciales y temporales). Supongamos que queremos tener simplemente 2 resoluciones espaciales $L^0_0$ y $L^0_1$. Como primero vamos a recibir $L^0_1$ y el bit-rate lo va a definir el usuario-contexto, por ejemplo, en un canal de TDT, 4 Mbps, el problema del rate allocation se nos ha acabado si en el canal de mejora (el que pasa de la resolución "1" a la "0") también transporta 4 Mbps. Si tenemos más niveles de resolución, el problema se resuelve trivialmente configurando HEVC para producir code-streams de 4 Mbps/canal. Ahora ... dentro del canal de mejora hay que cuantificar 

Si lo que queremos es escalar en calidad, de nuevo, la primera capa de calidad de $L^0_0$ debería comprimirse a 4 Mbps.

## More resolutions
PRMC se puede aplicar de forma recursiva a la secuencia $L^1$, y luego a $L^2$, etc.

## Sobre este notebook

Este notebook está [aquí](https://github.com/vicente-gonzalez-ruiz/PRMC-experimentos).